# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [15]:
lines = open("holbrook.txt").readlines()
data = []
# Write your code here
from nltk.tokenize import word_tokenize

for line in lines:
    original=[]
    corrected=[]
    indexes=[]
    tokenized=word_tokenize(line)
    #print(tokenized)
    for word_count,word in enumerate(tokenized):
        if '|' in word:
            original.append(word.split('|')[0])
            corrected.append(word.split('|')[1])
            indexes.append(word_count)
        else:
            original.append(word)
            corrected.append(word)
    data.append({'original':original,'corrected':corrected,'indexes':indexes})    
          
assert(data[2] == {
   'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
   'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
   'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [16]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [17]:
#Creating lits of all correct words
train_corrected=[letter['corrected'] for letter in train]

from collections import Counter
word_list=[]
for sentence in train_corrected:
    for i in sentence:
        word_list.append(i.lower())
        
def unigram(word):
    # Write your code here.
    word_freq=Counter(word_list)
    return word_freq[word]
    
def prob(word):
    # Write your code here.
    return (unigram(word)/len(word_list))

# Test your code with the following
assert(unigram("me")==87)

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [18]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("minde", "mind"))

1


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [19]:
unique_set=[]
for word_list in train_corrected:
    unique_set.extend(set(word_list))
unique_set=set(unique_set)
#     for each_word in word_list:
#         unique_set.add(each_word)
def get_candidates(token):
    # Write your code here.
    my_dict=dict()
    dist_list=[]
    for my_word in unique_set:
        my_dict[my_word]=edit_distance(token,my_word)
    min_distance=(min(my_dict.values()))
    for key,value in my_dict.items():
        if value==min_distance:
            dist_list.append(key)    
    return sorted(dist_list,reverse=True)
# Test your code as follows
assert get_candidates("minde") == ['mine', 'mind']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [20]:
def correct(sentence):
    # Write your code here
    final_sentence=[]
    tmp_dict=dict()
    tmp=[]
    for i in sentence:    
        if i not in unique_set:
            tmp=get_candidates(i)
            for j in tmp:
                tmp_dict[j]=prob(j)
            final_word=max(tmp_dict,key=tmp_dict.get)
            final_sentence.append(final_word)
        else:
            final_sentence.append(i)
    return final_sentence
# assert(correct(["this","whitr","cat"]) == ['this','white','cat']) 

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [21]:
def accuracy(test):
    # Write your code here
    original_list=[]
    corrected_list=[]
    for i in test:
        original_list.extend(i['original'])
        corrected_list.extend(i['corrected'])    
    correct_original_list=correct(original_list)       
    count=0
    for i in range(0,len(original_list)):
        if original_list[i]==correct_original_list[i]:
            count+=1
    return ((count/len(original_list))*100)
accuracy(test)

80.80985915492957

## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


In [22]:
from pyjarowinkler import distance
def jaro_distance(checker):
    j_dict=dict()
    j_list=[]
    for w in unique_set:
        if w != "":
            j_dict[w] = distance.get_jaro_distance(checker,w,winkler=True,scaling=0.1)
    max_distance=(max(j_dict.values()))
    for key,value in j_dict.items():
        if value==max_distance:
            j_list.append(key)    
    return j_list
# jaro_distance("mind")

**Approach**

The lecture videos had multiple n-grams and edit distance implementation was already employed in the algorithm.
Of the n-gram modelling, unigram probability seemed to be most suitable, so implemented the same along with jaro distance for finding word with minimum distance between two words.
This technique calculates jaro-winkler similarity between both the string and gives more accuracy when two strings start with the same character.
Jaro-Winker checks if the character of the string matches, order of the characters and gives similarity in the range from 0 to 1.
Result 0 shows no similarity and 1 shows that the strings are equal.
So, the list of words obtained from this algorithm are very less when compared to edit-distance as this algorithm gives continous values.

In [23]:
def jaro_correct(sentence):
    final_sentence=[]
    tmp_dict=dict()
    tmp=[]
    for i in sentence:    
        if i not in unique_set:
            tmp=jaro_distance(i)
            for j in tmp:
                tmp_dict[j]=prob(j)
            final_word=max(tmp_dict,key=tmp_dict.get)
            final_sentence.append(final_word)
        else:
            final_sentence.append(i)
    return final_sentence
#jaro_correct(["this","minde","cat"])

**Checking Jaro-Similarity**

In the above piece of code, I check if the word in the present in the cleaned training set.
If the word if not present, the word is considered to be incorrect.
Once the incorrect word is found, find the jaro-winkler similarity and check the unigram probability of the word. 
Finally, select the word with highest similarity and higher probabilty to replace the incorrect word.

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [25]:
def accuracy(test):
    original_list=[]
    corrected_list=[]
    for i in test:
        original_list.extend(i['original'])
        corrected_list.extend(i['corrected'])        
    original_list = [item.lower() for item in original_list]
    corrected_list = [item.lower() for item in corrected_list]
    correct_original_list=jaro_correct(original_list)
    correct_original_list = [item.lower() for item in correct_original_list]
    count=0
    for i in range(0,len(original_list)):
        if original_list[i]==correct_original_list[i]:
            count+=1
    return ((count/len(original_list))*100)
accuracy(test)

81.42605633802818

**Accuracy**

This implementation works best as the jaro-winkler algorithm reduces the number of word possibilty.
As the number of replacement words obtained is less, the probability of single word replacement is high.
Furthermore to improve the accuracy of the model, converting all words to lower and then compare to get more accurate results.
Accuracy of 80.8 from the earlier model has improved to 81.42 percent.
